# Django Excel App: Project Setup in JupyterLab


This notebook walks through creating a Django project that reads an Excel file and displays it in a web browser. It includes:
- Creating the Django project and app
- Setting up the view to read and render the Excel file
- Adding URL routing
- Creating a Bootstrap-styled HTML template


In [6]:
%cd C:\Users\pmuhuri\AppData\Local\anaconda3\envs\my_django_env


C:\Users\pmuhuri\AppData\Local\anaconda3\envs\my_django_env


In [ ]:
!pip install django openpyxl pandas

In [10]:

import os

project_root = "myproject"
app_name = "excelapp"
os.makedirs(os.path.join(project_root, "data"), exist_ok=True)
os.makedirs(os.path.join(project_root, app_name), exist_ok=True)
os.makedirs(os.path.join(project_root, app_name, "templates", app_name), exist_ok=True)
os.makedirs(os.path.join(project_root, "myproject"), exist_ok=True)


In [ ]:

import pandas as pd

df = pd.DataFrame({
    "Name": ["Alice", "Bob", "Charlie"],
    "Score": [85, 90, 95]
})
df.to_excel(os.path.join(project_root, "data", "combined_all.xlsx"), index=False)


In [ ]:

with open(os.path.join(project_root, "manage.py"), "w") as f:
    f.write("""#!/usr/bin/env python
import os
import sys

if __name__ == "__main__":
    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)
""")


In [ ]:

with open(os.path.join(project_root, "myproject", "settings.py"), "w") as f:
    f.write(f"""import os

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

SECRET_KEY = 'dummy-key'
DEBUG = True
ALLOWED_HOSTS = []

INSTALLED_APPS = [
    '{app_name}',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'myproject.urls'

TEMPLATES = [
    {{
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {{
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        }},
    }},
]

WSGI_APPLICATION = 'myproject.wsgi.application'

DATABASES = {{
    'default': {{
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }}
}}

LANGUAGE_CODE = 'en-us'
TIME_ZONE = 'UTC'
USE_I18N = True
USE_L10N = True
USE_TZ = True

STATIC_URL = '/static/'
""")


In [ ]:

with open(os.path.join(project_root, app_name, "views.py"), "w") as f:
    f.write("""import os
import pandas as pd
from django.shortcuts import render
from django.conf import settings

def excel_view(request):
    file_path = os.path.join(settings.BASE_DIR, 'data', 'combined_all.xlsx')
    df = pd.read_excel(file_path, engine='openpyxl')
    table_html = df.to_html(classes="table table-striped", index=False)
    return render(request, 'excelapp/excel.html', {'table_html': table_html})
""")


In [ ]:

# App-level urls.py
with open(os.path.join(project_root, app_name, "urls.py"), "w") as f:
    f.write("""from django.urls import path
from . import views

urlpatterns = [
    path('', views.excel_view, name='excel'),
]
""")

# Project-level urls.py
with open(os.path.join(project_root, "myproject", "urls.py"), "w") as f:
    f.write(f"""from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('', include('{app_name}.urls')),
    path('admin/', admin.site.urls),
]
""")


In [ ]:

open(os.path.join(project_root, "myproject", "__init__.py"), "w").close()

with open(os.path.join(project_root, "myproject", "wsgi.py"), "w") as f:
    f.write("""import os
from django.core.wsgi import get_wsgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
application = get_wsgi_application()
""")

with open(os.path.join(project_root, "myproject", "asgi.py"), "w") as f:
    f.write("""import os
from django.core.asgi import get_asgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
application = get_asgi_application()
""")


In [ ]:

html_path = os.path.join(project_root, app_name, "templates", app_name, "excel.html")
with open(html_path, "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Excel Data</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body>
    <div class="container mt-5">
        <h1>Excel File Contents</h1>
        {{ table_html|safe }}
    </div>
</body>
</html>
""")


### Run your Django development server using:

In [ ]:
!python myproject/manage.py runserver